In [ ]:
import cv2
import numpy as np
import time
import math

WIND_X = 1100
WIND_Y = 700

class Object:
    def __init__ (self, x_, y_, z_):
        self.x = x_
        self.y = y_
        self.z = z_
    
    def get_coords (self):
        return [self.x, self.y, self.z]
    
    def change_coord (self, coord, val, increment = False):
        if (coord == "x"):
            if (increment == True):
                self.x += val            
            else:
                self.x = val

        if (coord == "y"):
            if (increment == True):
                self.y += val            
            else:
                self.y = val

        if (coord == "z"):
            if (increment == True):
                self.z += val            
            else:
                self.z = val

    def draw (self, canvas):
        pass

class Light_emitter (Object):
    def __init__ (self, x_, y_, z_, color_ = (255, 255, 255)):
        Object.__init__ (self, x_, y_, z_)
        
        self.color = list(color_)
    
    def get_color (self):
        return self.color
    
    def set_color (self, new_val, channel):
        self.color[channel] = new_val
    
    def draw (self, canvas):
        pass
        #cv2.circle (canvas, (int (self.x / self.z), int (self.y / self.z)),
        #    10, (250, 250, 250), -1)

class Surface (Object):
    def __init__ (self, x_, y_, z_):
        Object.__init__ (x_, y_, z_)
    
    def draw (self, canvas):
        self.render (canvas)
    
    def render (self, canvas):
        pass
    
    def iterate_elements (self):
        pass
    
    def calc_element_lightening (self):
        pass
    
    def _dotproduct (self, v1, v2):
        return sum ((a * b) for a, b in zip (v1, v2))

    def _length(self, v):
        return math.sqrt (self._dotproduct (v, v))

    def _cos (self, v1, v2):
        return self._dotproduct (v1, v2) / (self._length (v1) * self._length (v2))
    
    def _subtr (self, v1, v2):
        return [a - b for a, b in zip (v1, v2)]

class Sphere(Surface):
    def __init__ (self, x_, y_, z_, r_, color_ = (100, 100, 255)):
        Object.__init__ (self, x_, y_, z_)
        
        self.r     = r_
        self.color = color_
    
    def iterate_elements (self):
        result = []
        
        for i in range (int (- self.r / self.z) * 2, int (self.r / self.z) * 2):
            for j in range (int (- self.r / self.z) * 2, int (self.r / self.z) * 2):
                dist_2d = self._length ((i, j))
                
                #if (dist_2d < self.r / self.z):
                n = []

                n.append (i * self.z)
                n.append (j * self.z)
                n.append (math.sqrt (abs ((self.r)**2 - (dist_2d * self.z)**2)))

                result.append ((self.x / self.z + i, self.y / self.z + j, n))

        return result
    
    def draw (self, canvas, emitter):
        self.render (canvas, emitter)
    
    def render (self, canvas, emitter):
        for i, j, n in self.iterate_elements ():
            pix_color = self.calc_lightening (n, emitter)
            
            def crop_val(val, lb, ub):
                if (val <= lb):
                    return lb
                
                if (val >= ub):
                    return ub
                
                return val
            
            canvas [crop_val (int (j), 0, WIND_Y - 1), crop_val(int (i), 0, WIND_X - 1),  :] = pix_color
    
    def calc_lightening (self, n, emitter):
        p_vec = [self.x + n [0], self.y + n [1], self.z + n [2]]
        vec = self._subtr (emitter.get_coords (), p_vec)
        cos = max (self._cos (n, vec), 0)
        #cos = self._cos (n, vec)
        
        em_color = emitter.get_color()
        
        result = [int (channel * cos * float (em_color [i]) / 255)
                  for channel, i in zip (self.color, range(3))]
        
        return result

emitter = Light_emitter (350, 200, 400)

cv2.namedWindow("render", cv2.WINDOW_AUTOSIZE)

spheres = [#Sphere (300, 350, 1, 250),
           Sphere (500, 450, 1, 250, color_ = (255, 255, 255)),
           #Sphere (650, 250, 1, 100, color_ = (25, 240, 10)),
           #Sphere (750, 150, 1, 100, color_ = (125, 200, 10)),
           #Sphere (850, 345, 1, 100, color_ = (225, 240, 10)),
           #Sphere (950, 125, 1, 100, color_ = (250, 20, 255))
           ]

to_refresh = True

def change_color(val, channel):
    global to_refresh
    
    emitter.set_color(val, channel)
    to_refresh = True

cv2.createTrackbar('B', 'render', 255, 255, lambda val : change_color(val, 0))
cv2.createTrackbar('G', 'render', 255, 255, lambda val : change_color(val, 1))
cv2.createTrackbar('R', 'render', 255, 255, lambda val : change_color(val, 2))

canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 55

while (True):
    k = cv2.waitKey (50) & 0xFF
    
    if (k != 255):
        to_refresh = True
    
    if (k == ord ('q')):
        break
    
    step = 20
    
    if (k == ord ('d')):
        emitter.change_coord ("x", step, increment = True)

    if (k == ord ('a')):
        emitter.change_coord ("x", -step, increment = True)

    if (k == ord ('s')):
        emitter.change_coord ("y", step, increment = True)

    if (k == ord ('w')):
        emitter.change_coord ("y", -step, increment = True)

    if (k == ord ('r')):
        emitter.change_coord ("z", 0.1, increment = True)

    if (k == ord ('f')):
        emitter.change_coord ("z", -0.1, increment = True)
    
    if (to_refresh == True):
        to_refresh = False
        canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 55
        
        print ("rendering...")
        
        before_time = time.time ()
        
        for sphere in spheres:
            sphere.draw (canvas, emitter)
        
        print ("rendered in ", str (time.time () - before_time) [:6], "seconds")
        
        emitter.draw (canvas)
    
    light_info_str  = "light: " + str (emitter.get_coords ())
    sphere_info_str = "sphere: " + str (sphere.get_coords ())
    
    canvas = cv2.putText (canvas, light_info_str, (20, 30),
        cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 25, 130), 1, cv2.LINE_AA)

    canvas = cv2.putText (canvas, sphere_info_str, (20, 60),
        cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 25, 130), 1, cv2.LINE_AA)
    
    cv2.imshow ("render", canvas)

#cv2.waitKey (0)
cv2.destroyAllWindows ()

rendering...
rendered in  7.6123 seconds
rendering...
rendered in  7.6719 seconds
rendering...
rendered in  7.5201 seconds
rendering...
rendered in  7.4604 seconds
rendering...
rendered in  7.5989 seconds
rendering...
rendered in  7.5182 seconds
rendering...
rendered in  7.5903 seconds
